Практическая работа №6. Атака по переносу (Transfer Attack) на модели ИИ

1. Загрузка и создание двух различных моделей

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model1 = Sequential([
  Flatten(input_shape=(28, 28)),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')])

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])

model1.fit(train_images, train_labels, epochs=5)

model1.save('mnist_model1.h5')

model2 = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')
])

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])

model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)

model2.save('mnist_model2.h5')

Epoch 1/5


c:\Users\ildar\nc\Уник\мага\2 курс\1 сем\АЗСИИ\repo\.venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 758us/step - accuracy: 0.8819 - loss: 0.4285
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 753us/step - accuracy: 0.9659 - loss: 0.1177
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 756us/step - accuracy: 0.9769 - loss: 0.0762
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - accuracy: 0.9837 - loss: 0.0535
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - accuracy: 0.9876 - loss: 0.0422


Epoch 1/5


c:\Users\ildar\nc\Уник\мага\2 курс\1 сем\АЗСИИ\repo\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9072 - loss: 0.3074
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9843 - loss: 0.0519
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9903 - loss: 0.0321
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9941 - loss: 0.0193
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9955 - loss: 0.0139


2. Реализация атаки FGSM на первую модель

In [2]:
import numpy as np

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
  perturbed_image = image + epsilon * np.sign(gradient)
  perturbed_image = np.clip(perturbed_image, 0, 1)
  return perturbed_image

def generate_adversarial_example(model, image, label, epsilon):
    image = tf.convert_to_tensor(image.reshape((1, 28, 28, 1)))

    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label)
    label = tf.convert_to_tensor(label)

    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)

    gradient = tape.gradient(loss, image)

    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())

    return np.reshape(adversarial_image, (28, 28, 1))

def generate_adversarial_dataset(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        adv_image = generate_adversarial_example(model, images[i], labels[i], epsilon)
        adversarial_images.append(adv_image.reshape(28, 28))

    adversarial_images = np.array(adversarial_images)

    print("Размерность adversarial_images:", adversarial_images.shape)

    return adversarial_images


# Генерация противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_adversarial_dataset(model1, test_images, test_labels, epsilon)

Размерность adversarial_images: (10000, 28, 28)


3. Оценка противоречивых примеров на обеих моделях

In [3]:
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Точность модели 1: {acc1}')

# Оценка второй модели на противоречивых примерах (перенос атаки)
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Точность модели 2: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step - accuracy: 0.0702 - loss: 6.4371
Точность модели 1: 0.094200000166893
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 877us/step - accuracy: 0.9583 - loss: 0.1384
Точность модели 2: 0.9656000137329102


4. Анализ переносимости атак

In [ ]:
adversarial_images_model2 = generate_adversarial_dataset(model2,
test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28, 28), test_labels)
print(f'Точность модели 1: {acc3}')

Размерность adversarial_images: (10000, 28, 28)
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step - accuracy: 0.9112 - loss: 0.2851
Точность модели 1: 0.9284999966621399


В ходе работы было изучено влияние атаки по переносу, при которой противоречивые примеры, созданные для одной модели, применялись для атаки на другую.

Результаты показали, что первая модель, для которой были сгенерированы примеры с использованием FGSM, испытала значительное падение точности, что свидетельствует о её высокой уязвимости к этой атаке. Вторая модель, напротив, продемонстрировала лишь незначительное снижение точности, что говорит о её большей устойчивости к переносу атак, созданных для другой модели.

Таким образом, атака по переносу с использованием FGSM может значительно снизить точность модели, особенно если примеры сгенерированы для неё самой. Однако модели с большей устойчивостью остаются менее восприимчивыми к таким атакам. Эти результаты подчёркивают необходимость разработки более защищённых моделей, способных сохранять точность даже при воздействии перенесённых атак.